<a href="https://colab.research.google.com/github/SalahAdDin/ThesisObjectDetectionDeepLearning/blob/master/RetinaNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Detection in Google Colab with Fizyr Retinanet

We are testing the RetinaNet technique against different datasets in order to prove which one has best performance detecting fruits in tree canopies.

It runs in Google Colab using [Fizyr implementation](https://github.com/fizyr/keras-retinanet) of RetinaNet in Keras.

Requirements are only dataset images and annotations file made in [LabelImg](https://github.com/tzutalin/labelImg).

Colab Runtime type: Python3, GPU enabled.

## Setup
Reviewing hardware setup:

In [0]:
%tensorflow_version 2.x

!lsb_release -a
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

tf.test.gpu_device_name()


No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.3 LTS
Release:	18.04
Codename:	bionic
TensorFlow version:  2.2.0
Num GPUs Available:  1


'/device:GPU:0'

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1569322203339851878, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12824497347737169434
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10052200176754359482
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15701463552
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11182901109516237562
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [0]:
!cat /proc/meminfo

MemTotal:       13333556 kB
MemFree:         9229564 kB
MemAvailable:   11870460 kB
Buffers:           74536 kB
Cached:          2685548 kB
SwapCached:            0 kB
Active:          1197948 kB
Inactive:        2484268 kB
Active(anon):     856168 kB
Inactive(anon):     8524 kB
Active(file):     341780 kB
Inactive(file):  2475744 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               920 kB
Writeback:             0 kB
AnonPages:        922148 kB
Mapped:           667148 kB
Shmem:              9140 kB
Slab:             174168 kB
SReclaimable:     128536 kB
SUnreclaim:        45632 kB
KernelStack:        3776 kB
PageTables:         8632 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666776 kB
Committed_AS:    3132432 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:              920 kB
AnonHugePages:   

In [0]:
!/usr/local/cuda/extras/demo_suite/deviceQuery

/usr/local/cuda/extras/demo_suite/deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla P100-PCIE-16GB"
  CUDA Driver Version / Runtime Version          10.1 / 10.1
  CUDA Capability Major/Minor version number:    6.0
  Total amount of global memory:                 16281 MBytes (17071734784 bytes)
  (56) Multiprocessors, ( 64) CUDA Cores/MP:     3584 CUDA Cores
  GPU Max Clock rate:                            1329 MHz (1.33 GHz)
  Memory Clock rate:                             715 Mhz
  Memory Bus Width:                              4096-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of constant memory:     

In [0]:
!sudo apt-get install -y --no-install-recommends libnvinfer6=6.0.1-1+cuda10.0 \
    libnvinfer-dev=6.0.1-1+cuda10.0 \
    libnvinfer-plugin6=6.0.1-1+cuda10.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libnvinfer-dev libnvinfer-plugin6 libnvinfer6
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 136 MB of archives.
After this operation, 460 MB of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  libnvinfer6 6.0.1-1+cuda10.0 [66.8 MB]
Get:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  libnvinfer-dev 6.0.1-1+cuda10.0 [67.3 MB]
Get:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  libnvinfer-plugin6 6.0.1-1+cuda10.0 [1,711 kB]
Fetched 136 MB in 2s (59.0 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <

In [0]:
!export PATH=/usr/local/cuda-10.0/bin:/usr/local/cuda-10.0/NsightCompute-1.0${PATH:+:${PATH}}
!export LD_LIBRARY_PATH=/usr/local/cuda-10.0/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}


Download and install in Colab required packages and import libraries:

**numpy**: updating the library.  
**gdown**: enables us to donwload heavy files from *Google Drive*.  

In [0]:
!pip install -U gdown #tensorflow numpy

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.11.0-cp36-none-any.whl size=9619 sha256=baf4891cc5a006186938e1455bbc923ac5bb355d5161dfbee4cba3dedea82a9d
  Stored in directory: /root/.cache/pip/wheels/19/a6/67/ebb80360fc57bb0ddd5da77f57b275084cd8838bf7d5b91685
Successfully built gdown
  Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninstalled gdown-3.6.4


In [0]:
#!git clone https://github.com/fizyr/keras-retinanet.git

In [0]:
!git clone https://github.com/SalahAdDin/keras-retinanet.git --branch exps --single-branch

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 5456 (delta 3), reused 2 (delta 0), pack-reused 5444
Receiving objects: 100% (5456/5456), 14.74 MiB | 32.67 MiB/s, done.
Resolving deltas: 100% (3643/3643), done.


In [0]:
%cd keras-retinanet/
!pip install .

/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=170390 sha256=286624598728033f4294de23a68becd211f92df4966e2f26815b8c41dc4ad831
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13346 sha256=696bc8153406df453bd5c1cebdbb20a4fbe147ceed8ab78f03568e744f082aae
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet keras-resnet


In [0]:
pip install -U keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [0]:
!python setup.py build_ext --inplace

running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/keras-retinanet/keras_retinanet/utils/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'keras_retinanet.utils.compute_overlap' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/keras_retinanet
creating build/temp.linux-x86_64-3.6/keras_retinanet/utils
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -c keras_retinanet/utils/compute_overlap.c -o build/temp.linux-x86_64-3.6/keras_retinanet/utils/compute_over

Updating **Keras**, **TensoFlow** requires the newest version.

In [0]:
import cv2; print ("OpenCV version :  {0}".format(cv2.__version__))
import keras; print("Keras version :  {0}".format(keras.__version__))
import tensorflow as tf; print("TensorFlow version :  {0}".format(tf.__version__))
import tensorboard as tb; print("TensorBoard version :  {0}".format(tb.version.VERSION))

OpenCV version :  4.1.2
Keras version :  2.3.1
TensorFlow version :  2.2.0
TensorBoard version :  2.2.1


Using TensorFlow backend.


## Making Dataset
Download from Drive training dataset.

In [0]:
import os
import shutil
import zipfile

from google.colab import drive
from google.colab import files

#### OPTIONAL: EXPORT TRAINED MODEL TO DRIVE ####
drive.mount('/content/gdrive', force_remount=False)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
DATASET_DIR = 'dataset'
CLASSES_FILE = '/content/keras-retinanet/classes.csv'

os.makedirs(DATASET_DIR, exist_ok=True)
os.makedirs('logs', exist_ok=True)

### MangoYOLO

In [0]:
# drive_url = 'https://drive.google.com/uc?id=' + DATASET_DRIVEID
file_name = 'MangoYOLO.zip'

!gdown https://drive.google.com/uc?id=1iZbPYck5BuIsZ9GQCSHI4LlqMyxmCrP9
# !gdown https://drive.google.com/uc?id=1oDOJFuhuixkx1ePu1G1HqSeKXhuhfA-B
print('Download completed!')

with zipfile.ZipFile(file_name, 'r') as zip_ref:
  zip_ref.extractall(DATASET_DIR)
os.remove(file_name)
print('Extract completed!')

Downloading...
From: https://drive.google.com/uc?id=1iZbPYck5BuIsZ9GQCSHI4LlqMyxmCrP9
To: /content/keras-retinanet/MangoYOLO.zip
163MB [00:02, 74.8MB/s]
Download completed!
Extract completed!


Concatenate the training and evaluation files according to the PASCAL VOC format:

In [0]:
# !cat /content/keras-retinanet/dataset/VOCDevkit/VOC2007/ImageSets/Main/train.txt /content/keras-retinanet/dataset/VOCDevkit/VOC2007/ImageSets/Main/val.txt > /content/keras-retinanet/dataset/VOCDevkit/VOC2007/ImageSets/Main/trainval.txt

### WGISD(grapes)

In [0]:
file_name = 'wgisd.zip'
!gdown https://drive.google.com/uc?id=1L2feEdder8H5U1G5FjH6lsx6u3yapXfx
print('Download completed!')

with zipfile.ZipFile(file_name, 'r') as zip_ref:
  zip_ref.extractall(DATASET_DIR+'/wgisd')
os.remove(file_name)
print('Extract completed!')

Downloading...
From: https://drive.google.com/uc?id=1L2feEdder8H5U1G5FjH6lsx6u3yapXfx
To: /content/keras-retinanet/wgisd.zip
301MB [00:05, 53.0MB/s]
Download completed!
Extract completed!


### ACFR Multifruit

In [0]:
file_name = 'acfr-fruit-dataset.zip'

!gdown https://drive.google.com/uc?id=1d0DXYLCAZHrL4RudWP_4Gty46MSRH4_G
print('Download completed!')

with zipfile.ZipFile(file_name, 'r') as zip_ref:
  zip_ref.extractall(DATASET_DIR)
os.remove(file_name)
print('Extract completed!')

Downloading...
From: https://drive.google.com/uc?id=1d0DXYLCAZHrL4RudWP_4Gty46MSRH4_G
To: /content/keras-retinanet/acfr-fruit-dataset.zip
2.23GB [00:33, 67.4MB/s]
Download completed!
Extract completed!


## Training Model
Download pretrained model and run training.

In the next cell choose one option:
1.   download Fizyr Resnet50 pretrained model
2.   download your custom pretrained model, to continue previous training epochs

In the last cell optionally export trained model to Google Drive.

In [0]:
"""
import urllib

PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

#### OPTION 1: DOWNLOAD INITIAL PRETRAINED MODEL FROM FIZYR ####
URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

#### OPTION 2: DOWNLOAD CUSTOM PRETRAINED MODEL FROM GOOGLE DRIVE. CHANGE DRIVE_MODEL VALUE. USE THIS TO CONTINUE PREVIOUS TRAINING EPOCHS ####
#drive.mount('/content/gdrive')
#DRIVE_MODEL = '/content/gdrive/My Drive/Colab Notebooks/objdet_tensorflow_colab/resnet50_csv_10.h5'
#shutil.copy(DRIVE_MODEL, PRETRAINED_MODEL)

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)
"""

"\nimport urllib\n\nPRETRAINED_MODEL = './snapshots/_pretrained_model.h5'\n\n#### OPTION 1: DOWNLOAD INITIAL PRETRAINED MODEL FROM FIZYR ####\nURL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'\nurllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)\n\n#### OPTION 2: DOWNLOAD CUSTOM PRETRAINED MODEL FROM GOOGLE DRIVE. CHANGE DRIVE_MODEL VALUE. USE THIS TO CONTINUE PREVIOUS TRAINING EPOCHS ####\n#drive.mount('/content/gdrive')\n#DRIVE_MODEL = '/content/gdrive/My Drive/Colab Notebooks/objdet_tensorflow_colab/resnet50_csv_10.h5'\n#shutil.copy(DRIVE_MODEL, PRETRAINED_MODEL)\n\nprint('Downloaded pretrained model to ' + PRETRAINED_MODEL)\n"

In [0]:
import os
import shutil
import zipfile
import urllib
import xml.etree.ElementTree as ET
import numpy as np
import csv
import pandas
import time

!mkdir /content/models

DRIVE_DIR = '/content/gdrive/My Drive/Colab Notebooks/'

Let's begin to train the model:

In [0]:
### OPTIONAL: Using transfer learning
# !keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --batch-size 8 --steps 500 --epochs 10 pascal /content/keras-retinanet/dataset/VOCDevkit/VOC2007

In [0]:
# MangoYOLO
start = time.time()
!keras_retinanet/bin/train.py --random-transform --batch-size 8 --steps 500 --epochs 10 pascal /content/keras-retinanet/dataset/VOCDevkit/VOC2007 > retinanet_mango_yolo.log
final = time.time() - start
print("Training time in seconds: ", final)
print("Training time in hours: ", final/(60*60))

!mv /content/keras-retinanet/snapshots/resnet50_pascal_10.h5 /content/models/resnet50_pascal_mango_yolo.h5
!rm -rf snapshots/*

shutil.copy('/content/models/resnet50_pascal_mango_yolo.h5', DRIVE_DIR)
shutil.copy('/content/keras-retinanet/retinanet_mango_yolo.log', DRIVE_DIR)

Using TensorFlow backend.
2020-05-12 09:15:01.598774: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-12 09:15:07.232428: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-12 09:15:07.235383: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-12 09:15:07.236015: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-05-12 09:15:07.236052: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-12 09:15:07.245209: I ten

'/content/gdrive/My Drive/Colab Notebooks/retinanet_mango_yolo.log'

In [0]:
# ACFR Almonds
start = time.time()
!keras_retinanet/bin/train.py --random-transform --batch-size 8 --steps 500 --epochs 10 pascal --image-extension '.png'  /content/keras-retinanet/dataset/acfr-fruit-dataset/almonds > retinanet_almonds.log
final = time.time() - start
print("Training time in seconds: ", final)
print("Training time in hours: ", final/(60*60))

!mv /content/keras-retinanet/snapshots/resnet50_pascal_10.h5 /content/models/resnet50_pascal_almonds.h5
!rm -rf snapshots/*

shutil.copy('/content/models/resnet50_pascal_almonds.h5', DRIVE_DIR)
shutil.copy('/content/keras-retinanet/retinanet_almonds.log', DRIVE_DIR)

Using TensorFlow backend.
2020-05-12 10:40:14.325912: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-12 10:40:20.189978: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-12 10:40:20.192440: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-12 10:40:20.193030: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-05-12 10:40:20.193064: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-12 10:40:20.201002: I ten

'/content/gdrive/My Drive/Colab Notebooks/retinanet_almonds.log'

In [0]:
# ACFR Apples
start = time.time()
!keras_retinanet/bin/train.py --random-transform --batch-size 8 --steps 500 --epochs 10 pascal --image-extension '.png' /content/keras-retinanet/dataset/acfr-fruit-dataset/apples > retinanet_apples.log
final = time.time() - start
print("Training time in seconds: ", final)
print("Training time in hours: ", final/(60*60))

!mv /content/keras-retinanet/snapshots/resnet50_pascal_10.h5 /content/models/resnet50_pascal_apples.h5
!rm -rf snapshots/*

shutil.copy('/content/models/resnet50_pascal_apples.h5', DRIVE_DIR)
shutil.copy('/content/keras-retinanet/retinanet_apples.log', DRIVE_DIR)

Using TensorFlow backend.
2020-05-12 11:53:32.300299: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-12 11:53:34.911564: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-12 11:53:34.916091: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-12 11:53:34.916705: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-05-12 11:53:34.916747: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-12 11:53:34.919041: I ten

'/content/gdrive/My Drive/Colab Notebooks/retinanet_apples.log'

In [0]:
# ACFR Mangoes
start = time.time()
!keras_retinanet/bin/train.py --random-transform --batch-size 8 --steps 500 --epochs 10 pascal --image-extension '.png' /content/keras-retinanet/dataset/acfr-fruit-dataset/mangoes > retinanet_mangoes.log
final = time.time() - start
print("Training time in seconds: ", final)
print("Training time in hours: ", final/(60*60))

!mv /content/keras-retinanet/snapshots/resnet50_pascal_10.h5 /content/models/resnet50_pascal_mangoes.h5
!rm -rf snapshots/*

shutil.copy('/content/models/resnet50_pascal_mangoes.h5', DRIVE_DIR)
shutil.copy('/content/keras-retinanet/retinanet_mangoes.log', DRIVE_DIR)

Using TensorFlow backend.
2020-05-12 13:39:46.723665: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-12 13:39:51.989547: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-12 13:39:51.992109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-12 13:39:51.992646: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-05-12 13:39:51.992679: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-12 13:39:52.001003: I ten

'/content/gdrive/My Drive/Colab Notebooks/retinanet_mangoes.log'

In [26]:
# WGISD
start = time.time()
!keras_retinanet/bin/train.py --random-transform --batch-size 8 --steps 500 --epochs 10 pascal /content/keras-retinanet/dataset/wgisd > retinanet_wgisd.log
final = time.time() - start
print("Training time in seconds: ", final)
print("Training time in hours: ", final/(60*60))

!mv /content/keras-retinanet/snapshots/resnet50_pascal_10.h5 /content/models/resnet50_pascal_wgisd.h5
!rm -rf snapshots/*

shutil.copy('/content/models/resnet50_pascal_wgisd.h5', DRIVE_DIR)
shutil.copy('/content/keras-retinanet/retinanet_wgisd.log', DRIVE_DIR)

Using TensorFlow backend.
2020-05-12 14:55:13.863582: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-12 14:55:19.923994: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-12 14:55:19.932804: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-12 14:55:19.933356: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-05-12 14:55:19.933387: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-12 14:55:19.939529: I ten

'/content/gdrive/My Drive/Colab Notebooks/retinanet_wgisd.log'

In [27]:
#### OPTIONAL: EXPORT TRAINED MODEL TO DRIVE ####
# drive.mount('/content/gdrive')
"""
COLAB_MODELS = [
               '/content/models/resnet50_pascal_mango_yolo.h5',
               '/content/models/resnet50_pascal_almonds.h5',
               '/content/models/resnet50_pascal_apples.h5',
               '/content/models/resnet50_pascal_mangoes.h5', 
               '/content/models/resnet50_pascal_wgisd.h5'
               ]
COLAB_LOGS = [
              '/content/keras-retinanet/retinanet_mango_yolo.log',
              '/content/keras-retinanet/retinanet_almonds.log',
              '/content/keras-retinanet/retinanet_apples.log',
              '/content/keras-retinanet/retinanet_mangoes.log', 
              '/content/keras-retinanet/retinanet_wgisd.log'
              ]

for model in COLAB_MODELS:
  try:
    shutil.copy(model, DRIVE_DIR)
  except:
    print('{} does not exist'.format(model))
for log in COLAB_LOGS:
  try:
    shutil.copy(log, DRIVE_DIR)
  except:
    print('{} does not exist'.format(log))
"""

"\nCOLAB_MODELS = [\n               '/content/models/resnet50_pascal_mango_yolo.h5',\n               '/content/models/resnet50_pascal_almonds.h5',\n               '/content/models/resnet50_pascal_apples.h5',\n               '/content/models/resnet50_pascal_mangoes.h5', \n               '/content/models/resnet50_pascal_wgisd.h5'\n               ]\nCOLAB_LOGS = [\n              '/content/keras-retinanet/retinanet_mango_yolo.log',\n              '/content/keras-retinanet/retinanet_almonds.log',\n              '/content/keras-retinanet/retinanet_apples.log',\n              '/content/keras-retinanet/retinanet_mangoes.log', \n              '/content/keras-retinanet/retinanet_wgisd.log'\n              ]\n\nfor model in COLAB_MODELS:\n  try:\n    shutil.copy(model, DRIVE_DIR)\n  except:\n    print('{} does not exist'.format(model))\nfor log in COLAB_LOGS:\n  try:\n    shutil.copy(log, DRIVE_DIR)\n  except:\n    print('{} does not exist'.format(log))\n"

In [0]:
%tensorboard --logdir logs

## Getting trained models

It is optional if we already have the trained models.

In [0]:
"""
%cd /content/models
!gdown https://drive.google.com/uc?id=1T221fzP2FjaKrA3-SjwdGUC2KYqr1It6
!gdown https://drive.google.com/uc?id=1OIuE5oj-R7GPDoKN8W3tKnw8733KgFDI
!gdown https://drive.google.com/uc?id=1-2CHYBJsmQHdWaPgyQpV86DvFyIIuRKZ
!gdown https://drive.google.com/uc?id=1rvsBrJFjKIDfybNvNvnJI-yw8SVp9SrT
!gdown https://drive.google.com/uc?id=1glvdm85qN4yGxXsgNRTlMT4ESa_s5LCJ
print('Download completed!')
%cd /content/keras-retinanet
"""

## Evaluation
Now, we will evaluate the trained model.

Optional: We can take the already trained model from Google Drive in order to avoid the training.

In [28]:
# MangoYOLO
!keras_retinanet/bin/evaluate.py --convert-model --iou-threshold 0.3 --save-path /content/keras-retinanet/images pascal /content/keras-retinanet/dataset/VOCDevkit/VOC2007/ /content/models/resnet50_pascal_mango_yolo.h5

Using TensorFlow backend.
2020-05-12 20:28:28.332929: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Loading model, this may take a second...
2020-05-12 20:28:31.350161: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-12 20:28:31.355310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-12 20:28:31.355895: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-05-12 20:28:31.355936: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.

In [29]:
# WGISD (Grapes)
!keras_retinanet/bin/evaluate.py --convert-model --iou-threshold 0.3 --save-path /content/keras-retinanet/images pascal /content/keras-retinanet/dataset/wgisd/ /content/models/resnet50_pascal_wgisd.h5

Using TensorFlow backend.
2020-05-12 20:29:31.172696: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Loading model, this may take a second...
2020-05-12 20:29:32.903854: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-12 20:29:32.906143: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-12 20:29:32.906688: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-05-12 20:29:32.906721: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.

In [30]:
# ACFR Fruit - Almonds 
!keras_retinanet/bin/evaluate.py --convert-model --iou-threshold 0.3 --save-path /content/keras-retinanet/images pascal --image-extension '.png' /content/keras-retinanet/dataset/acfr-fruit-dataset/almonds/ /content/models/resnet50_pascal_almonds.h5

Using TensorFlow backend.
2020-05-12 20:30:15.574390: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Loading model, this may take a second...
2020-05-12 20:30:17.291774: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-12 20:30:17.294060: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-12 20:30:17.294608: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-05-12 20:30:17.294642: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.

In [31]:
# ACFR Fruit - Apples
!keras_retinanet/bin/evaluate.py --convert-model --iou-threshold 0.3 --save-path /content/keras-retinanet/images pascal --image-extension '.png' /content/keras-retinanet/dataset/acfr-fruit-dataset/apples/ /content/models/resnet50_pascal_apples.h5

Using TensorFlow backend.
2020-05-12 20:30:53.118240: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Loading model, this may take a second...
2020-05-12 20:30:54.857934: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-12 20:30:54.860169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-12 20:30:54.860706: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-05-12 20:30:54.860739: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.

In [32]:
# ACFR Fruit - Mangoes
!keras_retinanet/bin/evaluate.py --convert-model --iou-threshold 0.3 --save-path /content/keras-retinanet/images pascal --image-extension '.png' /content/keras-retinanet/dataset/acfr-fruit-dataset/mangoes/ /content/models/resnet50_pascal_mangoes.h5

Using TensorFlow backend.
2020-05-12 20:31:32.981373: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Loading model, this may take a second...
2020-05-12 20:31:34.995885: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-12 20:31:34.998172: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-12 20:31:34.998735: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-05-12 20:31:34.998781: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.

## Inference
Run inference with uploaded image on trained model.  

*Optional*: We can take the already trained model from *Google Drive* in order to avoid the training.

In [0]:
THRES_SCORE = 0.8

In [0]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%reload_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

In [0]:
model_path = os.path.join('/content/models', sorted(os.listdir('/content/models'), reverse=True)[0])
print(model_path)

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

# load label to names mapping for visualization purposes
labels_to_names = pandas.read_csv(CLASSES_FILE,header=None).T.loc[0].to_dict()

# print(labels_to_names)

In [0]:
def img_inference(img_path):
  image = read_image_bgr(img_infer)

  # copy to draw on
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  # preprocess image for network
  image = preprocess_image(image)
  image, scale = resize_image(image)

  # process image
  start = time.time()
  boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
  print("processing time: ", time.time() - start)

  # correct for image scale
  boxes /= scale

  # visualize detections
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
      # scores are sorted so we can break
      if score < THRES_SCORE:
          break

      color = label_color(label)

      b = box.astype(int)
      draw_box(draw, b, color=color)

      caption = "{} {:.3f}".format(labels_to_names[label], score)
      draw_caption(draw, b, caption)

  plt.figure(figsize=(10, 10))
  plt.axis('off')
  plt.imshow(draw)
  plt.show()

In [0]:
uploaded = files.upload()
img_infer = list(uploaded)[0]

print('Running inference on: ' + img_infer)
img_inference(img_infer)